In [ ]:
# This notebook is trying out different pipeline approaches

# Cropped data is used here (5 minutes only), tried on whole data - takes forever.


#Load data, make folders
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
import pyprep as pp

In [ ]:
#from Functions.main_meg_qc import initial_stuff


#Not working:
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
#(calculates for 5h to no results - 2nd tutorial)

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003352/sub-1/ses-01/meg/sub-1_ses-01_task-ColorSpirals_run-00_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003352/sub-4/ses-02/meg/sub-4_ses-02_task-ColorSpirals_run-01_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003392/sub-01/meg/sub-01_task-localizer_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003645/sub-002/meg/sub-002_task-FacePerception_run-1_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003694/sub-02/meg/sub-02_task-MEM_run-01_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif'
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004276/sub-002/meg/sub-002_task-words_meg.fif'

#WORKING 1st tutorial only:
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003703/sub-a68d5xp5/meg/sub-a68d5xp5_task-listeningToSpeech_run-01_meg.fif'

#WORKING 1st tutorial only:
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003682/sub-001/ses-01/meg/sub-001_ses-01_task-AversiveLearningReplay_run-01_meg.fif'


raw = mne.io.read_raw_fif(data_file, on_split_missing='ignore')

#crop the data to calculate faster
raw_cropped = raw.copy()
#raw_cropped.crop(tmin=1100, tmax=None) 

raw_cropped

In [ ]:
# Annotate movement - like in tutorial 2 from:
# https://mne.tools/stable/generated/mne.preprocessing.annotate_movement.html

from mne.preprocessing import annotate_movement, compute_average_dev_head_t

# Get cHPI time series and compute average
chpi_locs = mne.chpi.extract_chpi_locs_ctf(raw)
head_pos = mne.chpi.compute_head_pos(raw.info, chpi_locs)
original_head_dev_t = mne.transforms.invert_transform(
    raw.info['dev_head_t'])
average_head_dev_t = mne.transforms.invert_transform(
    compute_average_dev_head_t(raw, head_pos))
fig = mne.viz.plot_head_positions(head_pos)
for ax, val, val_ori in zip(fig.axes[::2], average_head_dev_t['trans'][:3, 3],
                            original_head_dev_t['trans'][:3, 3]):
    ax.axhline(1000 * val, color='r')
    ax.axhline(1000 * val_ori, color='g')

# The green horizontal lines represent the original head position, whereas the
# red lines are the new head position averaged over all the time points.

mean_distance_limit = 0.0015  # in meters
annotation_movement, hpi_disp = annotate_movement(
    raw, head_pos, mean_distance_limit=mean_distance_limit)
raw.set_annotations(annotation_movement)
raw.plot(n_channels=100, duration=20)

In [ ]:
# try another tutorial: 
# (takes some minutes depends how big is raw_cropped: 2-40 min):
# https://mne.tools/stable/auto_tutorials/preprocessing/59_head_positions.html#sphx-glr-auto-tutorials-preprocessing-59-head-positions-py

# cHPI - continuous head position indicator (HPI) coil channels, data in teslas

# 'We can use mne.chpi.get_chpi_info to retrieve the coil frequencies, the index of 
# the channel indicating when which coil was switched on, and the respective “event codes” 
# associated with each coil’s activity.'
chpi_freqs, ch_idx, chpi_codes = mne.chpi.get_chpi_info(info=raw_cropped.info, on_missing='warn', verbose=None)
# Output:
# - The frequency used for each individual cHPI coil.
# - The index of the STIM channel containing information about when which cHPI coils were switched on.
# - The values coding for the “on” state of each individual cHPI coil.

# The values coding for the “on” state of each individual cHPI coil.
print(f'cHPI coil frequencies extracted from raw: {chpi_freqs} Hz')

#We only got 5, not 9 HPI (see error in cell above)


#extract the HPI coil amplitudes as a function of time:
print('Extract the HPI coil amplitudes as a function of time:')
chpi_amplitudes=mne.chpi.compute_chpi_amplitudes(raw_cropped)
#chpi_amplitudes=mne.chpi.compute_chpi_amplitudes(raw_cropped, t_step_min=0.01, t_window='auto', ext_order=1, tmin=0, tmax=None, verbose=None)


#compute time-varying HPI coil locations from these
print('Compute time-varying HPI coil locations from these')
#chpi_locs=mne.chpi.compute_chpi_locs(raw_cropped.info, chpi_amplitudes, t_step_max=1.0, too_close='raise', adjust_dig=False, verbose=None)
chpi_locs=mne.chpi.compute_chpi_locs(raw_cropped.info, chpi_amplitudes)


print('Compute head positions from the coil locations:')
#compute head positions from the coil locations:
head_pos = mne.chpi.compute_head_pos(raw_cropped.info, chpi_locs, verbose=True)
print('head_positions computed:', head_pos)

In [ ]:
#Visualizing continuous head position: doesnt work

mne.viz.plot_head_positions(head_pos, mode='traces')
#mne.viz.plot_head_positions(head_pos, mode='field')

In [ ]:
# Now again try to annotate movement: also error - It didnt calculate any positions!

mean_distance_limit = 0.0015  # in meters
annotation_movement, hpi_disp = annotate_movement(
    raw_cropped, head_pos, mean_distance_limit=mean_distance_limit)
raw_cropped.set_annotations(raw_cropped.annotations + annotation_movement)
raw_cropped.plot(n_channels=100, duration=20)